In [1]:
from PIL import Image
import os
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from einops import rearrange, einsum

In [2]:
import maskclip_onnx

def load_clip():
    """
    Load CLIP and its preprocessing transforms.
    Cache it, so we only have to do it once.
    """
    model, preprocess = maskclip_onnx.clip.load(
        name='ViT-L/14@336px', device=torch.device("cuda")
    )
    return model, preprocess

/home/roger/anaconda3/envs/hdt/lib/python3.8/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/roger/anaconda3/envs/hdt/lib/python3.8/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [3]:
model, preprocess = load_clip()

device=torch.device("cuda")

In [4]:
import torch.nn as nn

class export_wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, images):
        aspect_ratio = images.shape[-1] / images.shape[-2]
        target_H = 336
        target_W = int(target_H * aspect_ratio)
        images = F.interpolate(images, size=(target_H, target_W), mode='bilinear', align_corners=False)
        # normalize
        rgb_mean = torch.tensor((0.48145466, 0.4578275, 0.40821073)).reshape(1, 3, 1, 1).to('cuda')
        rgb_std  = torch.tensor((0.26862954, 0.26130258, 0.27577711)).reshape(1, 3, 1, 1).to('cuda')
        images = (images - rgb_mean) / rgb_std

        # Get CLIP embeddings for the images
        # with logger.time("get_clip_embeddings"):
        patch_embeddings = model.get_patch_encodings(images)
        # patch_embeddings = []
        # patch_embeddings.append(model.get_patch_encodings(preprocessed_images))
        # patch_embeddings = torch.cat(patch_embeddings, dim=0)

        # Reshape embeddings to number of patches in height and width
#         h_in, w_in = images.shape[-2:]
#         h_out = h_in // model.visual.patch_size
#         w_out = w_in // model.visual.patch_size
#         embeddings = rearrange(patch_embeddings, "b (h w) d -> b d h w", h=h_out, w=w_out)
        return patch_embeddings

In [5]:
my_export_model = export_wrapper(model)

In [8]:
image_tensors = torch.randn(1, 3, 240, 320).to(device)

In [9]:
input_tensor = image_tensors[0].unsqueeze(0)

input_tensor = input_tensor.float().cuda()
print(input_tensor.shape)
print(input_tensor.max())
print(input_tensor.min())
my_export_model = my_export_model.float()

torch.Size([1, 3, 240, 320])
tensor(4.6410, device='cuda:0')
tensor(-5.2805, device='cuda:0')


In [10]:
torch.onnx.export(my_export_model, input_tensor, 'exported_clip.onnx', export_params=True)

/tmp/ipykernel_14924/236815134.py:11: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  target_W = int(target_H * aspect_ratio)
/tmp/ipykernel_14924/236815134.py:14: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  rgb_mean = torch.tensor((0.48145466, 0.4578275, 0.40821073)).reshape(1, 3, 1, 1).to('cuda')
/tmp/ipykernel_14924/236815134.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables 

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [27]:
raw_descriptor = my_export_model(input_tensor)
print(raw_descriptor.shape)
print(raw_descriptor.max())
print(raw_descriptor.min())
print(raw_descriptor.mean())

torch.Size([1, 1008, 768])
tensor(14.6705, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-13.8217, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.0029, device='cuda:0', grad_fn=<MeanBackward0>)


In [29]:
import time

start_cp = time.time()

for _ in range(1000):
    ret = my_export_model(input_tensor)

end_cp = time.time()

print("Total time: {}".format(end_cp - start_cp))
print("FPS: {}".format(1000 / (end_cp - start_cp)))

Total time: 64.9676730632782
FPS: 15.392270537779687
